In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

import pandas as pd
import numpy as np

# fix random seed for reproducibility
tf.random.set_seed(1337)
np.random.seed(1337)

In [2]:
# load dataset and scramble its rows
data = pd.read_csv('./imdb_processed.csv')
data = data.iloc[np.random.permutation(len(data))]

In [9]:
# TODO: SHOULD DO THIS ONLY ON THE TRAINING DATA

from collections import Counter
from tqdm import tqdm
tqdm.pandas()

# # get all processed reviews
# reviews = data.text.values
# # merge into single variable, separated by whitespaces
# words = ' '.join(reviews)
# # obtain list of words
# words = words.split()

words = data['text'].str.cat(sep=' ').split()

# build vocabulary
frequency_counter = Counter(words)
# sort words by the frequency they appear in the text
vocab = sorted(frequency_counter, key=frequency_counter.get, reverse=True)

# associate a number to each word in the list in ascending order
# in this way the most frequent words have lower numbers
int2word = dict(enumerate(vocab[:5000], 2))
int2word[0] = '<PAD>'
int2word[1] = '<UNK>'
word2int = {word: id for id, word in int2word.items()}
# encode words
reviews_enc = [[word2int.get(word, '<UNK>') for word in review.split()] for review in tqdm(data.text.values)]

with open('tmp.txt', 'w') as file:
  for el in reviews_enc:
    file.write(str(el) + '\n')  # Convert the list to a string and add a newline character

print("saved to txt")

100%|██████████| 50000/50000 [00:00<00:00, 58871.52it/s]


saved to txt


In [ ]:
train_size = 0.8
validation_size = 0.5

split_id = int(len(data) * train_size)

features = reviews_enc
labels = data['label']

temp_train_x, test_x = features[:split_id], features[split_id:]
temp_train_y, test_y = labels[:split_id], labels[split_id:]

# make val and test set
split_val_id = int(len(temp_train_x) * validation_size)
train_x, val_x = temp_train_x[:split_val_id], temp_train_x[split_val_id:]
train_y, val_y = temp_train_y[:split_val_id], temp_train_y[split_val_id:]

In [ ]:
max_review_length = 500
X_train = sequence.pad_sequences(train_x, maxlen=max_review_length)
X_test = sequence.pad_sequences(test_x, maxlen=max_review_length)

In [ ]:
print(len(counter))
print(len(vocab))

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=3, batch_size=64)